In [1]:
from input_preprocess import tokenizer
from lwm_model import lwm
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from input_preprocess import create_labels
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns

# 选择数据集
scenario_names = np.array([
    "city_18_denver", "city_15_indianapolis", "city_19_oklahoma",
    "city_12_fortworth", "city_11_santaclara", "city_7_sandiego"
])
scenario_idxs = np.array([0, 1, 2, 3, 4, 5])  # Select the scenario indexes
selected_scenario_names = scenario_names[scenario_idxs]

# 对数据进行标记
preprocessed_chs = tokenizer(
    selected_scenario_names=selected_scenario_names,  # Selects predefined DeepMIMOv3 scenarios_test. Set to None to load your own dataset.
    manual_data=None,  # If using a custom dataset, ensure it is a wireless channel dataset of size (N,32,32) based on the settings provided above.
    gen_raw=True  # Set gen_raw=False to apply masked channel modeling (MCM), as used in LWM pre-training. For inference, masking is unnecessary unless you want to evaluate LWM's ability to handle noisy inputs.
)

# 加载模型
# 这里应为mps加速和cuda加速存在兼容问题(float类型不兼容)，所以在这里无法使用mps
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading the LWM model on {device}...")
model = lwm.from_pretrained(device=device)

# 进行推理
from inference import lwm_inference, create_raw_dataset
input_types = ['cls_emb', 'channel_emb', 'raw']
selected_input_type = input_types[2]  # Change the index to select LWM CLS embeddings, LWM channel embeddings, or the original input channels.

if selected_input_type in ['cls_emb', 'channel_emb']:
    dataset = lwm_inference(preprocessed_chs, selected_input_type, model, device)
else:
    dataset = create_raw_dataset(preprocessed_chs, device)


Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_18_denver/BS3_UE_0-6970.mat


Generating channels: 100%|██████████| 6970/6970 [00:00<00:00, 51596.28it/s]



Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_15_indianapolis/BS3_UE_0-6320.mat


Generating channels: 100%|██████████| 6320/6320 [00:00<00:00, 19776.51it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_19_oklahoma/BS1_UE_0-6150.mat


...


Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_11_santaclara/BS1_UE_0-5358.mat


Generating channels: 100%|██████████| 5358/5358 [00:00<00:00, 21018.93it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_7_sandiego/BS1_UE_0-5893.mat


Processing items: 100%|██████████| 14840/14840 [00:00<00:00, 41997.69it/s]
/Users/dianhongyang/Desktop/毕业设计/LWM_Model/LWM/lwm_model.py:122: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimenta

Loading the LWM model on cpu...
Model loaded successfully from /Users/dianhongyang/Desktop/毕业设计/LWM_Model/LWM/model_weights.pth to cpu
torch.Size([14840, 128, 16])


In [2]:
dataset.shape

torch.Size([14840, 128, 16])

In [3]:
#获得labels
tasks = ['LoS/NLoS Classification', 'Beam Prediction']
task = tasks[1] # Choose 0 for LoS/NLoS labels or 1 for beam prediction labels.
labels = create_labels(task, scenario_names, n_beams=16) # For beam prediction, n_beams specifies the number of beams in the codebook. If you're generating labels for LoS/NLoS classification, you can leave this value unchanged as it doesn't impact the label generation.


Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_18_denver/BS3_UE_0-6970.mat


Computing the channel for each user: 100%|██████████| 6970/6970 [00:00<00:00, 234410.72it/s]



Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_15_indianapolis/BS3_UE_0-6320.mat


Computing the channel for each user: 100%|██████████| 6320/6320 [00:00<00:00, 111279.03it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_19_oklahoma/BS1_UE_0-6150.mat


...

Computing the channel for each user: 100%|██████████| 6192/6192 [00:00<00:00, 195226.15it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_11_santaclara/BS1_UE_0-5358.mat


Computing the channel for each user: 100%|██████████| 5358/5358 [00:00<00:00, 121664.86it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_7_sandiego/BS1_UE_0-5893.mat


Computing the channel for each user: 100%|██████████| 5893/5893 [00:00<00:00, 160418.97it/s]


In [4]:
# 每个信道是128 * 16的矩阵
# 可以使用卷积神经网络处理
# 数据划分
# 1. 信道数据

# 初始数据划分
x_train, x_test, y_train, y_test = train_test_split(
    dataset, labels, test_size=0.2, random_state=42, stratify=labels
)

# 二次划分：将训练集划分为训练集和验证集，比例为8:2
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
x_val.shape


torch.Size([2375, 128, 16])

In [5]:
# 数据归一化

from sklearn.preprocessing import StandardScaler
import torch

# 假设 x_train, x_val, x_test 是 Tensor，形状为 (num_samples, num_patches, patch_dim)
x_train_np = x_train.reshape(-1, x_train.shape[-1]).numpy()  # 形状变为 (num_samples * num_patches, patch_dim)
x_val_np = x_val.reshape(-1, x_val.shape[-1]).numpy()
x_test_np = x_test.reshape(-1, x_test.shape[-1]).numpy()

# 使用 StandardScaler 对所有补丁的特征进行归一化
scaler = StandardScaler()
scaler.fit(x_train_np)  # 只用训练集的统计信息
x_train_scaled = scaler.transform(x_train_np).reshape(x_train.shape)  # 还原为 (num_samples, num_patches, patch_dim)
x_val_scaled = scaler.transform(x_val_np).reshape(x_val.shape)
x_test_scaled = scaler.transform(x_test_np).reshape(x_test.shape)

# 转换回 Tensor
x_train = torch.tensor(x_train_scaled, dtype=torch.float32)
x_val = torch.tensor(x_val_scaled, dtype=torch.float32)
x_test = torch.tensor(x_test_scaled, dtype=torch.float32)


y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [6]:
batch_size = 128
# 定义 DataLoader
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 可选：查看各部分数据的大小
print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")
print(f"测试集大小: {len(test_dataset)}")

训练集大小: 9497
验证集大小: 2375
测试集大小: 2968


In [7]:
import torch
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, patch_dim, num_patches, num_classes, embed_dim=64, num_heads=4, num_layers=2, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        
        # 补丁嵌入层
        self.embedding = nn.Linear(patch_dim, embed_dim)  # 将每个补丁从 patch_dim 映射到 embed_dim
        
        # Transformer 编码器层
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,  # 输入的维度
            nhead=num_heads,    # 多头注意力的头数
            dim_feedforward=embed_dim * 4,  # 前馈网络的维度
            dropout=dropout
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 分类头
        self.fc = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
        # x: (batch_size, num_patches, patch_dim)
        x = self.embedding(x)   # 映射到 (batch_size, num_patches, embed_dim)
        x = x.permute(1, 0, 2)  # 转置为 (num_patches, batch_size, embed_dim)
        x = self.encoder(x)     # Transformer 编码器
        x = x.mean(dim=0)       # 对序列（补丁）取平均，得到 (batch_size, embed_dim)
        x = self.fc(x)          # 分类层
        return x

# 模型初始化
patch_dim = 16  # 每个补丁的维度
num_patches = 128  # 补丁的数量
num_classes = 16  # 分类的类别数
model = TransformerClassifier(patch_dim, num_patches, num_classes)
print(model)


TransformerClassifier(
  (embedding): Linear(in_features=16, out_features=64, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=64, out_features=16, bias=True)
)


In [8]:
device =  torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

TransformerClassifier(
  (embedding): Linear(in_features=16, out_features=64, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=64, out_features=16, bias=True)
)

In [9]:
# 定义训练参数
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score, accuracy_score


epochs = 200
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 开始训练
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)  # 前向传播
        loss = criterion(outputs, batch_y)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == batch_y).sum().item()
        total += batch_y.size(0)

    train_accuracy = train_correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    # 验证集评估
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(batch_y.cpu().numpy())

    # 计算验证集指标
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average="weighted")

    print(f"Accuracy: {val_accuracy:.4f}  F1-Score: {val_f1:.4f}")

# 训练集评估
train_preds = []
train_labels = []

with torch.no_grad():
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(batch_y.cpu().numpy())

# 计算训练集指标
train_accuracy = accuracy_score(train_labels, train_preds)
train_f1 = f1_score(train_labels, train_preds, average="weighted")
train_recall = recall_score(train_labels, train_preds, average="weighted")
train_conf_matrix = confusion_matrix(train_labels, train_preds)

print("\nTraining Metrics:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1-Score: {train_f1:.4f}")
print(f"Recall: {train_recall:.4f}")

# 绘制混淆矩阵
plt.figure(figsize=(12, 8))
sns.heatmap(train_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Training Confusion Matrix")
plt.show()

Epoch 1/200, Loss: 195.7344, Accuracy: 0.1696
Accuracy: 0.1848  F1-Score: 0.1014
Epoch 2/200, Loss: 192.6241, Accuracy: 0.1775
Accuracy: 0.1789  F1-Score: 0.0856
Epoch 3/200, Loss: 191.1192, Accuracy: 0.1872
Accuracy: 0.1836  F1-Score: 0.0787
Epoch 4/200, Loss: 189.7248, Accuracy: 0.1884
Accuracy: 0.1794  F1-Score: 0.0998
Epoch 5/200, Loss: 185.8332, Accuracy: 0.2006
Accuracy: 0.1933  F1-Score: 0.1052


KeyboardInterrupt: 